In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def decorate(html_spans):
    spans = []
    
    for i in range(len(html_spans)):
        coordinates = []
        coordinates.append(html_spans[i][0])
        coordinates.append(html_spans[i][1])
        spans.append(coordinates)
        
    #contents_without_attributes = fun_output[1]
    #spans_in_html_spans = fun_output[2]
    #texts = fun_output[3]
    #attributes = getattr(text,layer).attributes
    #contents = []
    
    #for content in contents_without_attributes:
    #    new_content=[]
    #    for i in range(len(content)):
    #        new_content.append(attributes[i%len(attributes)])
    #        new_content.append(content[i])
    #    contents.append(new_content)
    
    #tabledata=[]
    
    #for content in contents_without_attributes:
    #    for i in range(len(attributes)):
    #        tabledata.append(attributes[i])
    #        tabledata.append(content[i])
    
    texts = []
    
    for span in html_spans:
        content=[]
        span_info = span[2]
        for i in range(len(span_info)):
            content.append(span_info[i].text)
        texts.append(content)
    
    fragments = []
    last_end = 0
    print(texts)
    style='background-color:blue;'
    
    for i, (s, e) in enumerate(spans):
        fragments.append(text[last_end:s])
        fragments.append('<span class="span')
        if len(html_spans[i][2])>1:
            fragments.append(" overlapping-span")
        fragments.append('"; onclick="visualise{0}(')
        fragments.append(str(html_spans[i][1]).replace('"','').replace('{','{{').replace('}','}}'))
        fragments.append(',')
        fragments.append(str(i))
        fragments.append(',')
        fragments.append("["+str(html_spans[i][0])+","+str(html_spans[i][1])+"]")
        fragments.append(", this,")
        fragments.append(str(texts[i]))
        fragments.append(')"; indexes="')
        fragments.append("["+str(html_spans[i][0])+","+str(html_spans[i][1])+"]")
        fragments.append('"; span_exists = "";')
        fragments.append('style = "')
        fragments.append(style)
        fragments.append('">')
        fragments.append(text[s:e])
        fragments.append('</span>')
        last_end = e
    fragments.append(text[last_end:])
    return ''.join(fragments)

In [3]:
def is_pure_text(segment):
    for span in segment:
        if len(segment[1])>0:
            return False
    return True

In [4]:
def test_bg_mapping(segment):
    if 'KÜMME' in segment[1][0].attr_1:
        return "green"
    else:
        return "yellow"

In [5]:
def test_class_mapping(segment):
    return "some_class"

In [6]:
from estnltk.tests import new_text

In [7]:
def bg_mapper_discrete(segment, attribute, value_mapping, default_color, conflict_color):
    
    if segment[1].length() > 1:
        return conflict_color
    
    return value_mapping.get(segment[1][0].get_attr(attribute), default_color)    

In [8]:
def test_color_mapping(segment):
    return "blue"

In [9]:
def test_morph_bg_mapping(segment):
    if 'V' in segment[1][0].partofspeech:
        return "green"
    else:
        return "yellow"

In [10]:
from estnltk.visualisation.fancy_span_visualisation import DisplaySpans

display = DisplaySpans(styling="indirect")

In [11]:
display.span_decorator.update_css("prettyprinter.css")

In [12]:
def class_mapper(segment):
    if len(segment[1]) > 1:
        return "overlapping-span"
    return "span"

In [13]:
display.span_decorator.class_mapping=class_mapper

In [14]:
display(new_text(5).layer_1)

['<style>\n.span {\n    background-color: yellow;\n}\n\n.overlapping-span {\n    background-color: red;\n}\n\n.spanline {{\n    background-color: blue;\n    position: relative;\n    height: 3px;\n    margin-left: 0px;\n}}\n\n.tables {{\n    position: absolute;\n    width: fit-content;\n    width: -moz-fit-content;\n    border: 1px solid black;\n}}\n\n.maintext{0} {{\n    position: relative;\n}}\n\n.tables tbody tr:nth-child(even) {{\n    background-color: lightgray;\n}}\n\n.tables tbody tr:nth-child(odd) {{\n    background-color: beige;\n}}\n\n.tables tbody tr:hover {{\n    background-color: ivory;\n}}\n</style><span class=span >Sada</span>', ' ', '<span class=overlapping-span >kaks</span>', '<span class=overlapping-span >kümme</span>', '<span class=span >nd</span>', ' ', '<span class=span >kolm</span>', '.', '<span class=span > Neli</span>', ' ', '<span class=span >tuhat</span>', ' ', '<span class=overlapping-span >viis</span>', '<span class=overlapping-span >sada</span>', ' ', '<span

Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd tuhat.

In [15]:
display = DisplaySpans(styling="direct")

In [16]:
display.span_decorator.bg_mapping=test_bg_mapping

In [17]:
display.span_decorator.colour_mapping=test_color_mapping

In [18]:
display(new_text(5).layer_1)

['<span style=color:blue;background:yellow;>Sada</span>', ' ', '<span style=color:blue;background:yellow;>kaks</span>', '<span style=color:blue;background:green;>kümme</span>', '<span style=color:blue;background:green;>nd</span>', ' ', '<span style=color:blue;background:yellow;>kolm</span>', '.', '<span style=color:blue;background:yellow;> Neli</span>', ' ', '<span style=color:blue;background:yellow;>tuhat</span>', ' ', '<span style=color:blue;background:yellow;>viis</span>', '<span style=color:blue;background:yellow;>sada</span>', ' ', '<span style=color:blue;background:yellow;>kuus</span>', '<span style=color:blue;background:green;>kümme</span>', '<span style=color:blue;background:green;>nd</span>', ' ', '<span style=color:blue;background:yellow;>seitse</span>', ' ', '<span style=color:blue;background:yellow;>koma</span>', ' ', '<span style=color:blue;background:yellow;>kaheksa</span>', '. ', '<span style=color:blue;background:yellow;>Üheksa</span>', '<span style=color:blue;backgroun

Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd tuhat.